# TP1 - Pandas y Visualización de datos

P33. Estime la probabilidad de que, al seleccionar un rating al azar con y sin reposición, este usuario comparta alguna película vista con el 179792 que sea del mismo género que la película a la que le dejó rating.

In [2]:
import pandas as pd

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
ratings_original = pd.read_csv("/content/drive/MyDrive/Organización de Datos/TP1/Movie dataset/ratings.csv")
peliculas_original = pd.read_csv("/content/drive/MyDrive/Organización de Datos/TP1/Movie dataset/movies.csv")

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [87]:
pelis = peliculas_original[ ["id", "title", "genres"] ].dropna()
pelis["id"] = pelis["id"].astype(int)
ratings = ratings_original[ ["userId", "movieId", "rating"] ].dropna()
ratings = ratings.rename(columns={"movieId": "id"})

ratings_179792 = ratings[ratings["userId"] == 179792]
pelis_179792 = pelis.merge(ratings_179792, on="id", how="inner")

In [88]:
# Sin reposición

casos_totales = 5000 #Con 1000 elementos tarda aprox 58 segundos, con 5000 elementos tarda 4:30 minutos
muestra = ratings.sample(casos_totales, replace=False)

casos_favorables = 0

for usuario in muestra["userId"]:
  id_peli_rating = muestra[muestra["userId"] == usuario]["id"]
  genero_peli_rating = pelis[pelis["id"].isin(id_peli_rating)]["genres"]
  ratings_usuario = ratings[ratings["userId"] == usuario]
  pelis_usuario = pelis.merge(ratings_usuario, on="id", how="inner")
  pelis_en_comun = pelis_179792.merge(pelis_usuario, on=["id", "title", "genres"], how = "inner")
  if (genero_peli_rating.isin(pelis_en_comun["genres"])).any():
    casos_favorables += 1

probabilidad_sin_repo = casos_favorables / casos_totales
probabilidad_sin_repo

0.4364

In [85]:
# Con reposición

casos_totales = 5000
muestra = ratings.sample(casos_totales, replace=True)

casos_favorables = 0

for usuario in muestra["userId"]:
  id_peli_rating = muestra[muestra["userId"] == usuario]["id"]
  genero_peli_rating = pelis[pelis["id"].isin(id_peli_rating)]["genres"]
  ratings_usuario = ratings[ratings["userId"] == usuario]
  pelis_usuario = pelis.merge(ratings_usuario, on="id", how="inner")
  pelis_en_comun = pelis_179792.merge(pelis_usuario, on=["id", "title", "genres"], how = "inner")
  if (genero_peli_rating.isin(pelis_en_comun["genres"])).any():
    casos_favorables += 1

probabilidad_con_repo = casos_favorables / casos_totales
probabilidad_con_repo

0.4546